In [ ]:
!pip install openai==1.13.3

In [ ]:
# Add Azure OpenAI package
from openai import AzureOpenAI
from openai import AsyncAzureOpenAI

import asyncio
import requests

In [ ]:
user_prompt = input('Ask your medical concern: ')

Text Generation

In [ ]:
#Set the API Keys, Endpoint and Model Deployment

# Configuration settings
azure_oai_endpoint = "https://eyuser21.openai.azure.com/"
azure_oai_key = "5db726a49fb143ebb039e7c9a265b43b"
azure_oai_deployment = "chatbot0393"

# Initialize the Azure OpenAI client
client = AzureOpenAI(
        azure_endpoint = azure_oai_endpoint, 
        api_key=azure_oai_key,  
        api_version="2024-02-15-preview"
        )


print("Azure OpenAI client configured.")

# Generate Text

def generate_text(prompt = user_prompt):

    # To maintain chat hisotry, we initialize messages array and append all interactions to the messages array
    messages_array = [ 
        {"role": "system", "content": "You are a medical assistant and will provide medical opinions only."},

        # Negative example for non-medical question
        {"role": "user", "content": "Do penguins like pizzas?"},
        {"role": "assistant", "content": "This is not a medical question. Please ask a medical-related question."},
        
        # Positive example for symptom checking
        {"role": "user", "content": "I've been feeling really tired lately and I'm not sure why."},
        {"role": "assistant", "content": "I'm sorry to hear that you're not feeling well. Fatigue can be caused by a variety of factors. Have you experienced any other symptoms, or has there been a change in your daily routine that might be affecting your energy levels?"},

        # Positive example for symptom checking
        {"role": "user", "content": "I've been feeling really tired lately and I'm not sure why."},
        {"role": "assistant", "content": "Feeling tired can be normal. Maybe you just need some rest."},

        # Negative example for non-medical location question
        {"role": "user", "content": "Where is the nearest Starbucks?"},
        {"role": "assistant", "content": "I'm here to assist with medical inquiries."},

         # Positive example for medical advice on lifestyle recommendations
        {"role": "user", "content": "What kind of diet should I follow to lower my cholesterol?"},
        {"role": "assistant", "content": "A diet rich in fruits, vegetables, whole grains, nuts, fish, and lean meats can help lower cholesterol. Reducing intake of saturated fats, trans fats, and dietary cholesterol is also beneficial. Consider speaking with a dietitian for personalized advice."},

        # Negative example for non-medical entertainment question
        {"role": "user", "content": "Can you recommend a good movie to watch?"},
        {"role": "assistant", "content": "As a medical assistant, I'm here to provide medical opinions and information."},

        # Positive example for medical clarification
        {"role": "user", "content": "What does it mean if my blood test shows high potassium levels?"},
        {"role": "assistant", "content": "High potassium levels in a blood test can indicate hyperkalemia, which can be due to various conditions including kidney disease, dehydration, or certain medications. It's important to discuss these results with your healthcare provider for proper evaluation and management."},

        # Negative example for technical support question
        {"role": "user", "content": "My phone won't turn on, what should I do?"},
        {"role": "assistant", "content": "I specialize in medical assistance. "},

        # Positive example for health behavior inquiry
        {"role": "user", "content": "How many hours of sleep should an adult get each night?"},
        {"role": "assistant", "content": "Most adults need 7 to 9 hours of sleep per night. Getting enough sleep is crucial for good health, and it helps with mental and physical recovery."},

        # Negative example for travel recommendation
        {"role": "user", "content": "What's the best place to visit in Europe during the summer?"},
        {"role": "assistant", "content": "I'm here to help with medical questions."},

        # User prompt:
        {"role": "user", "content": prompt}
    ]

    response = client.chat.completions.create(
    model=azure_oai_deployment,
    temperature=0.7,
    max_tokens=1200,
    messages=messages_array
    )
    generated_text = response.choices[0].message.content
    # Add generated text to messages array
    messages_array.append({"role": "system", "content": generated_text})

    # return generated text
    return "Summary: " + generated_text + "\n"

Assistance = generate_text("I'm dowm with fever and cough. What should I do?")
image_prompt = Assistance.split("Summary:")[1]


Generate Image using the prompt and chat output

In [ ]:
#Set the API Keys, Endpoint and Model Deployment
azure_oai_endpoint ="https://eyuser26.openai.azure.com/"
azure_oai_key ="98138d9733f746a6aa3652f29c183d0d"
azure_oai_deployment ="Dalle3"

# Initialize the Azure OpenAI client
azure_endpoint = azure_oai_endpoint
api_key=azure_oai_key
api_version="2024-02-15-preview"

def generate_image(prompt = user_prompt, chat_prompt = image_prompt):

    prompt = "Generate medical images, for educational illustrations. " + prompt + chat_prompt

    # Call the DALL-E model
    url = "{}openai/deployments/dalle3/images/generations?api-version={}".format(azure_endpoint, api_version)
    headers= { "api-key": api_key, "Content-Type": "application/json" }
    body = {
                "prompt": prompt,
                "n": 1,
                "size": "1024x1024"
            }

    response = requests.post(url, headers=headers, json=body)
    print(response.text)

    # Get the revised prompt and image URL from the response
    revised_prompt = response.json()['data'][0]['revised_prompt']
    image_url = response.json()['data'][0]['url']

    # Display the URL for the generated image
    print(revised_prompt)
    print(image_url)

    save_path = 'prompt_image.jpg'

    # Download the image and save it to the specified path
    response = requests.get(image_url)
    if response.status_code == 200:
        with open(save_path, 'wb') as f:
            f.write(response.content)
        print(f"Image saved to {save_path}")
    else:
        print("Failed to download the image")
